In [15]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import sys
import glob
import math
import torchvision.transforms as transforms
from PIL import Image,resize

In [39]:
import models

In [16]:
train_root = "/home/yulin/Documents/DLCV/hw2/hw2_train_val/train15000/"
test_root = "/home/yulin/Documents/DLCV/hw2/hw2_train_val/val1500/"

In [26]:
class MYDATASET(Dataset):
    def __init__(self, root, transform=None):
        """ Intialize the MNIST dataset """
        self.images = None
        self.labels = None
        self.filenames = []
        self.root = root
        self.transform = transform
        #     parent = os.path.join(path, i)　　read filenames
        path = root+"images/"
        self.filenames=sorted(glob.glob(os.path.join(path,'*.jpg')))
        self.len = len(self.filenames)
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        image_fn, label = self.filenames[index]
        image = Image.open(image_fn).convert('RGB')
        image = Image.resize((448, 448),Image.ANTIALIAS)
        path_ = root+'labelTxt_hbb/'
        '''txtnames=sorted(glob.glob(os.path_.join(path_,'*.txt'))
        img = img.resize((width, height),Image.ANTIALIAS)'''
        f = open(path_ +str(index)+'.txt')
        line = f.readline()
        list = []
        while line:
            a = line.split( )
            b = a[0:]
            list.append(b)
            line = f.readline()
        f.close
        label_one=np.zeros((7,7,26))
        for s in range(len(list)):
            x=0.5*(float(list[s][2])+float(list[s][0]))/512
            y=0.5*(float(list[s][5])+float(list[s][1]))/512
            w=1*(float(list[s][2])-float(list[s][0]))/512
            h=1*(float(list[s][5])-float(list[s][1]))/512
            class_=list[s][8]
            diff_=list[s][9]
            i=math.floor(7*x)
            j=math.floor(7*y)
            label_one[i][j][0]=x
            label_one[i][j][1]=y
            label_one[i][j][2]=w
            label_one[i][j][3]=h
            label_one[i][j][4]=1
            if class_==str('plane'):
                label_one[i][j][10]=1
            if class_==str('ship'):
                label_one[i][j][11]=1
            if class_==str('storage-tank'):
                label_one[i][j][12]=1
            if class_==str('baseball-diamond'):
                label_one[i][j][13]=1
            if class_==str('tennis-court'):
                label_one[i][j][14]=1
            if class_==str('basketball-court'):
                label_one[i][j][15]=1
            if class_==str('ground-track-field'):
                label_one[i][j][16]=1
            if class_==str('harbor'):
                label_one[i][j][17]=1
            if class_==str('bridge'):
                label_one[i][j][18]=1
            if class_==str('small-vehicle'):
                label_one[i][j][19]=1
            if class_==str('large-vehicle'):
                label_one[i][j][20]=1
            if class_==str('helicopter'):
                label_one[i][j][21]=1
            if class_==str('roundabout'):
                label_one[i][j][22]=1
            if class_==str('soccer-ball-field'):
                label_one[i][j][23]=1
            if class_==str('swimming-pool'):
                label_one[i][j][24]=1
            if class_==str('container-crane'):
                label_one[i][j][25]=1
        label = torch.from_numpy(label_one)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return self.len

In [18]:
trainset=MYDATASET(root=train_root, transform=transforms.ToTensor())
testset=MYDATASET(root=test_root, transform=transforms.ToTensor())

In [19]:
print('# images in trainset:', len(trainset)) # Should print 15000
print('# images in testset:', len(testset)) # Should print 1500

# images in trainset: 15000
# images in testset: 1500


In [40]:
models = Yolov1_vgg16bn(pretrained=True)

In [12]:
"""path=test_lab_path
txtnames=sorted(glob.glob(os.path.join(path,'*.txt')))
for temp in txtnames:
    f = open(temp)
    line = f.readline()
    list = []
    while line:
        a = line.split( )
        b = a[0:]
        list.append(b)
        line = f.readline()
    f.close
print(txtnames)"""

"path=test_lab_path\ntxtnames=sorted(glob.glob(os.path.join(path,'*.txt')))\nfor temp in txtnames:\n    f = open(temp)\n    line = f.readline()\n    list = []\n    while line:\n        a = line.split( )\n        b = a[0:]\n        list.append(b)\n        line = f.readline()\n    f.close\nprint(txtnames)"

In [ ]:
'''objectness 分数的计算也使用 sigmoid 函数'''